In [ ]:
import os
from datetime import datetime

import torch
from torch import nn
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm

from classes.model import Net
from classes.model_version import ModelVersion
from classes.earlystopper import EarlyStopper
from functions import generate_model_name


# Control Parameters

In [ ]:
# Model and Dataset names
model_name = generate_model_name()
dataset_name = 'test'

# Hyperparameters
hyper_params = {
    'model_name': model_name,
    'train_date': datetime.now(),
    'dataset_name': dataset_name,
    'subset': 1,
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'batch_size': 128,
    'learning_rate': 1e-5,
    'input_size': 300, 
    'hidden_size': [512,1024,1024,512],
    'output_size': 10,
    'drop_out': 0.50,
    'num_epochs': 5000,
    'max_patience': 100,
    'lr_on': False,
    'step_size': False,
    'gamma': False,
}


# Model Version
version = ModelVersion(hyper_params)

# Earlystopper
earlystopper = EarlyStopper(max_patience=hyper_params['max_patience'], model_name=hyper_params['model_name'])

# Model
model = Net(hyper_params)
model = model.to(hyper_params['device'])


print(f'Model name: {model_name}')
print('Device: ' + str(hyper_params['device']))

## Training Function

In [ ]:
def train(model, version, earlystopper, hyper_params):
    
    # msg
    print('Currently training: ' + hyper_params['model_name'])
    
    # init  
    loss_fn = nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr = hyper_params['learning_rate'])
    tensorboard = SummaryWriter(log_dir = os.getcwd()+ '/runs/' + hyper_params['model_name'])
    dataloaders = get_dataset(dataset_name = hyper_params['dataset_name'],
                              subset = hyper_params['subset'],
                              batch_size = hyper_params['batch_size'])

    # iterate epochs
    for epoch in tqdm(range(hyper_params['num_epochs']), position=0, leave=True):
        
        # swithc phases
        for phase in ['train', 'val']:
                           
            # init
            ep_losses = []
            running_corrects = 0
            running_total = 0
            
            # set model mode
            if phase == 'train': model.train()
            else: model.eval()
                
            # iterate batches
            for batch in dataloaders[phase]:
                
                # Get inputs + labels
                x, y = batch
                
                # To gpu
                x = x.to(hyper_params['device'])
                y = y.to(hyper_params['device'])                

                # if train phase, track gradient
                with torch.set_grad_enabled(phase == 'train'):
                    
                    # Compute model ouput
                    optimizer.zero_grad()
                    y_hat = model(x)               

                    # Compute loss
                    loss = loss_fn(y_hat, y) 

                    # Backward + optimize if training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    ep_losses.append(loss.item())

                    # Accuracy
                    running_corrects += (torch.argmax(y_hat, axis=1) == y).sum().item() 
                    running_total += len(y)
                    
            # LR Decay                    
            if (hyper_params['lr_on'] == True) and (phase == 'train'):
                lrScheduler.step()

            # Compute mean loss and accuracy
            mean_loss = np.mean(ep_losses)
            acc = running_corrects/running_total 
            
            # Log to version object
            version.loss[phase].append(mean_loss)
            version.acc[phase].append(acc)
            
            # Log to tensorboard
            tensorboard.add_scalar(f'{phase}/Loss {phase}', mean_loss, epoch)
            tensorboard.add_scalar(f'{phase}/Accuracy {phase}', acc, epoch)
            
            # Early check
            if phase == 'val':
                stop_check = earlystopper.check_early_stop(loss=mean_loss, model=model, version=version)
                
                if stop_check:
                    print('Earlystopper stopped training!')
                    return min(version.loss['val'])

    return min(version.loss['val'])

## Train Model!

In [ ]:
loss =train(model=model, 
            version=version,
            earlystopper=earlystopper,
            hyper_params=hyper_params)